该脚本作用：获取Redis数据库中的任务并将其实现

In [63]:
# 获取电脑信息
import uuid
import socket
import platform

mac = uuid.UUID(int=uuid.getnode() ).hex[-12:]

##################查看用户主机名和IP地址
# 获取主机名
hostname = socket.gethostname()
# 获取IP
ip = socket.gethostbyname(hostname)
 
print("主机名：", hostname)
print("IP：", ip)

################查看用户电脑系统类型
system_type = platform.system()
print("Windows平台：", system_type)

主机名： Ming-Log
IP： 192.168.0.162
Windows平台： Windows


In [9]:
import redis
import requests

In [2]:
# 连接Redis数据库
redis_conn = redis.Redis(host="192.168.128.128", port=6379, password="minglog")

# 岗位信息任务

## 获取任务

In [6]:
job_id = redis_conn.spop('all_job_list').decode()

In [7]:
print(job_id)

1482193656365580295


## 获取数据

In [8]:
# 二级页面API
job_secondPage_api = 'https://www.5iai.com/api/enterprise/job/public'

In [10]:
job_secondPage_params = {
    'id': job_id
}

In [11]:
job_secondPage_response = requests.get(job_secondPage_api, params=job_secondPage_params)

In [12]:
job_secondPage_dict = job_secondPage_response.json()
if 'data' in job_secondPage_dict.keys():
    # 获取岗位技能
    job_skillsList = ','.join([i['labelName'] for i in job_secondPage_dict['data']['skillsList']])
    # 获取工作地点
    job_address = job_secondPage_dict['data']['enterpriseAddress']['detailedAddress']
    # 获取岗位职责
    job_Requiredments = job_secondPage_dict['data']['jobRequiredments']
else:
    # 如果没有岗位详细信息，即二级页面为空
    job_skillsList = ''
    job_address = ''
    job_Requiredments = ''

## 存储到数据库中

In [25]:
import pymysql
# 连接数据库
conn = pymysql.connect(host='localhost',
                      port=3306,
                      database='tipdm_cup',
                      user='root',
                      passwd='123456')
# 创建游标
cursor = conn.cursor()

In [50]:
job_sql =  "update job_1 set job_skillsList='%s', job_address='%s', job_Requiredments='%s' where job_id='%s';" % (job_skillsList, job_address, job_Requiredments, job_id)

In [51]:
cursor.execute(job_sql)

1

In [53]:
conn.commit()

## 写成函数

In [60]:
import redis
import pymysql
import uuid
import socket
import platform

In [ ]:
def get_job_info():
    # 连接Redis数据库
    redis_conn = redis.Redis(host="192.168.128.128", port=6379, password="minglog")
    # 连接数据库
    conn = pymysql.connect(host='localhost',
                          port=3306,
                          database='tipdm_cup',
                          user='root',
                          passwd='123456')
    # 创建游标
    cursor = conn.cursor()
    # 获取相关设备信息
    mac = uuid.UUID(int=uuid.getnode() ).hex[-12:]
    # 获取主机名
    hostname = socket.gethostname()
    # 获取IP
    ip = socket.gethostbyname(hostname)
    # 系统版本
    system_type = platform.system()
    while True:
        if redis_conn.scard('all_job_list') != 0:
            # 获取任务
            job_id = redis_conn.spop('all_job_list').decode()
            print(f"IP: {ip}, MAC:{mac}[{system_type}]: {job_id}")
            # 二级页面API
            job_secondPage_api = 'https://www.5iai.com/api/enterprise/job/public'
            # 二级页面参数
            job_secondPage_params = {
            'id': job_id
            }
            # 获取二级页面响应
            job_secondPage_response = requests.get(job_secondPage_api, params=job_secondPage_params)
            job_secondPage_dict = job_secondPage_response.json()
            # 解析二级页面响应
            if 'data' in job_secondPage_dict.keys():
                # 获取岗位技能
                job_skillsList = ','.join([i['labelName'] for i in job_secondPage_dict['data']['skillsList']])
                # 获取工作地点
                job_address = job_secondPage_dict['data']['enterpriseAddress']['detailedAddress']
                # 获取岗位职责
                job_Requiredments = job_secondPage_dict['data']['jobRequiredments']
            else:
                # 如果没有岗位详细信息，即二级页面为空
                job_skillsList = ''
                job_address = ''
                job_Requiredments = ''
            # 生成sql命令
            job_sql =  "update job_1 set job_skillsList='%s', job_address='%s', job_Requiredments='%s' where job_id='%s';" % (job_skillsList, job_address, job_Requiredments, job_id)
            # 运行sql命令
            cursor.execute(job_sql)
            # 提交sql命令
            conn.commit()
        else:
            # 关闭redis连接
            redis_conn.close()
            # 关闭mysql连接
            cursor.close()
            conn.close()
            break